<h1>Importing resources<h1>



1.   defs.csv
2.   stop words file



In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string #per punteggiatura
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from numpy import mean
import pandas as pd


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df = pd.read_csv (r'defs.csv')


def get_definition_for_concept(concept):
    concept_array= df[[concept]].to_numpy()
    
    concept_list=[]
    for arr in concept_array:
      concept_list.append(str(arr[0]))
    return concept_list

courage_definitions=get_definition_for_concept("Courage") #astratto generico

paper_definitions=get_definition_for_concept("Paper")
apprehension_definitions=get_definition_for_concept("Apprehension")

sharpener_definitions=get_definition_for_concept("Sharpener")

print(courage_definitions)
print(type(courage_definitions))

['nan', 'Ability to face our own fears and do something that scars us or makes us unpleasent', 'the ability to face thing without fear ', 'Inner strength thaht allow you to face particular situations', 'Ability to control the fear', 'nan', 'Ability to avoid fear and to take on risky actions', 'nan', 'Being able to do something fearful', 'the ability to do something despite being frightened.', 'nan', 'Feeling that allows us to face situations considered dangerous', 'the ability to do something that may be scary', 'is the ability to make drastic choices', 'ability to overcome fear', 'characteristic of a person who taking a risk', 'the quality of being able to do things which are generally dangerous or scaring', 'behavior typical of a hero', 'Ability to face difficult situations', 'ability to do something most people fear', 'Strength of mind that allows you to face difficult situations', "Ability to face one's fears", 'The ability to not be blocked by fear', 'emotion that allows someone t

In [ ]:
stop_words_list = []
lemmatizer = WordNetLemmatizer() 

'''
Lemmatization is the process of converting a word to its base form. 
The difference between stemming and lemmatization is, lemmatization considers 
the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, 
often leading to incorrect meanings and spelling errors.
'''

'\nLemmatization is the process of converting a word to its base form. \nThe difference between stemming and lemmatization is, lemmatization considers \nthe context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, \noften leading to incorrect meanings and spelling errors.\n'

In [ ]:
 nltk.download('tagsets') 
#decommentare per vedere tagset completo nltk 
#(adverb(RB),adverb comparative(RBR), adverbe superlative(RBS), adj(JJ), adj superlative(JJS) ecc.. )
#nltk.help.upenn_tagset()

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

Data una parola word, la tokenizzo con nltk.word_tokenize, che ritorna una lista di token.
Successivamente nltk.pos_tag ritorna una lista di tuple (word,tag), di cui 
io voglio solo il tag del primo elemento (ce ne è uno solo trattandosi di una sola parola e non di una frase), quindi vi accedo con [0][1] (cioè il secondo elemento del primo elemento della lista).
In seguito ritorno il tag wordnet in base al tag nltk


In [ ]:
# Ritorna il POS TAG di una parola
def get_wordnet_pos(word):
    '''
    nltk.pos_tag(nltk.word_tokenize(word)) ha questo risultato

    [('Ability', 'NN')]
    [('to', 'TO')]
    [('face', 'NN')]
        ....
        ....
        ....
    Volgio solo determinare il POS TAG di una parola, quindi: nltk.pos_tag(nltk.word_tokenize(word))[0][1]
    '''
    treebank_tag = nltk.pos_tag(nltk.word_tokenize(word))[0][1] 
    if treebank_tag.startswith('J'):# JJ: adjective or numeral, ordinal JJR: adjective, comparative
        return wn.ADJ
    elif treebank_tag.startswith('V'): #VB: verb, base form VBN: verb, past participle VBP: verb, present tense, not 3rd person singular VBZ: verb, present tense, 3rd person singular 
        return wn.VERB
    elif treebank_tag.startswith('N'): #NN: noun, common, singular or mass NNP: noun, proper, singular NNPS: noun, proper, plural NNS: noun, common, plural
        return wn.NOUN
    elif treebank_tag.startswith('R'): # RB: adverb RBR: adverb, comparative RBR: adverb, comparative RP: particle
        return wn.ADV
    else:
        return ''


In [ ]:
# Ritorna una lista con le stop words
def get_stop_words():
    if len(stop_words_list) == 0:
       
        f = open("stop_words_FULL.txt", "r") 
        for x in f:
            stop_words_list.append(x)
    # aggiunge l'apostrofo alla lista delle spot words in modo da ignorarlo
    return stop_words_list + ['’']


# Da una frase ritorna una lista con le singole parole (lemmi) rimuovendo le parole inutili (stop words..)
def get_list_of_gains_words(sentence):
    list_words_lemma = []
    # aus_list_words = sentence.split()
    aus_list_words = word_tokenize(sentence) #tokenizziamo la frase
    stop_words_list = get_stop_words() #prendo le stop words  dal file
    #per ogni parola nella lista dei token
    for w in aus_list_words:
        if w.lower() not in stop_words_list:#se la parola non è una stopword
            pos_tag = get_wordnet_pos(w) #prendo il post_tag di quella parola
            if pos_tag != '':
                #lemmatize può prendere come paramentro anche il pos tag per restringere il campo di ricerca del lemma
                list_words_lemma.append(lemmatizer.lemmatize(w.lower(), pos_tag)) 

    return list_words_lemma

# Ritorna la lista delle parole della signature di un synset lemmatizzata e senza stop words : gloss(definizione) + examples
def get_signature_of_synset(synset):
    # Gloss
    signature = get_list_of_gains_words(synset.definition()) #definizione senza stop words e con i lemmi
    # Examples
    for ex in synset.examples():
        list_ex = get_list_of_gains_words(ex) #esempio senza stop words e con i lemmi
        signature.extend(list_ex) # aggiunge l'esempio alla signature

    return signature


# Lemmatizzazione e rimozione stop word
def preprocessing():
    for i in range(len(courage_definitions)):
        courage_definitions[i] = bag_of_words(get_list_of_gains_words(courage_definitions[i]))
    
    for i in range(len(paper_definitions)):
        paper_definitions[i] = bag_of_words(get_list_of_gains_words(paper_definitions[i]))
    for i in range(len(sharpener_definitions)):
        sharpener_definitions[i] = bag_of_words(get_list_of_gains_words(sharpener_definitions[i]))
    for i in range(len(apprehension_definitions)):
        apprehension_definitions[i] = bag_of_words(get_list_of_gains_words(apprehension_definitions[i]))
    

# Restitusice l'insieme delle parole utilizzate nelle definizioni di
# un concetto (paper, sharpener, courage, apprehension)
def word_set(definizioni):
    words = set()

    for defin in definizioni:
        words.update(bag_of_words(defin))
        #for word in defin:
        #words.add(word)

    return words


# Data una lista di parole restituisce l'insieme delle parole (set) togliendo le parole ripetute
def bag_of_words(sentence):
    bag = set()
    for word in sentence:
        bag.add(word)
    return bag


# Dato un insieme di definizioni ne calcola il valore di similarità
def definitions_similarity(definitions):
    similarity = 0
    for i in range(len(definitions)):
        for j in range(len(definitions)):
            if i != j:
                similarity += set_similarity(definitions[i], definitions[j])

    n = len(definitions) * len(definitions) - len(definitions)
    return similarity / n


# Restituisce la similarità tra due definizione mediante cardinalità dell’intersezione dei termini
# normalizzata su lunghezza minima tra le due)
"""
es prof slide 19 lez 1.2
abcde
abdef     b,e   2/4=0,5 dove 4 è la lunghezza media delle definizioni di una parola ( ma noi usiamo la lunghezza minima)
bef
"""
def set_similarity(def1: set, def2: set):#gli passiamo delle definizioni che trasformiamo in set
    return len(def1.intersection(def2)) / min(len(def1), len(def2))




In [ ]:
#
# MAIN
#

preprocessing()

print("Astratto - Generico: ", definitions_similarity(courage_definitions))
print("Astratto - Specifico: ", definitions_similarity(apprehension_definitions))
print("Concreto - Generico: ", definitions_similarity(paper_definitions))
print("Concreto - Specifico: ", definitions_similarity(sharpener_definitions))
print()

#########################################################################################
#########################################################################################
#                   UTILIZZO DI WORDNET                                                 #
# Viene calcolata la similarità tra l'insieme delle parole utilizzate nelle             #
# definizioni di ciascun concetto e l'insieme delle parole utilizzate nella glossa e    #
# negli esempi del corrispondente WordNet synset                                        #
#########################################################################################
#########################################################################################

# Prendo gli insiemi delle parole utilizzate nelle gloss e nelle definizioni (signature)
# dei termini paper, sharpener, courage, apprehension
print("Con utilizzo di gloss e esempi di WordNet:")
paper_synset = wn.synsets('paper')[0] #prende il primo synset di wordnet per ogni concetto
sharpener_synset = wn.synsets('sharpener')[0]
courage_synset = wn.synsets('courage')[0]
apprehension_synset = wn.synsets('apprehension')[0]

paper_signature = set(get_signature_of_synset(paper_synset))
sharpener_signature = set(get_signature_of_synset(sharpener_synset))
courage_signature = set(get_signature_of_synset(courage_synset))
apprehension_signature = set(get_signature_of_synset(apprehension_synset))

# Prendo gli insiemi delle parole utilizzate nelle NOSTRE definizioni
# dei termini paper, sharpener, courage, apprehension
bulding_words_set = word_set(paper_definitions)
sharpener_words_set = word_set(sharpener_definitions)
courage_words_set = word_set(courage_definitions)
apprehension_words_set = word_set(apprehension_definitions)

# Calcolo similarità tra definizioni ( come set di parole depuarte e lemmatizzate) nostre e quelle di WordNet  (cardinalità dell’intersezione dei termini
# normalizzata su lunghezza minima tra le due)
astratto_generico = len(courage_words_set.intersection(courage_signature)) / min(len(courage_words_set),
                                                                                  len(courage_signature))
astratto_specifico = len(apprehension_words_set.intersection(apprehension_signature)) / min(len(apprehension_words_set),
                                                                                        len(apprehension_signature))
concreto_generico = len(bulding_words_set.intersection(paper_signature)) / min(len(bulding_words_set),
                                                                                  len(paper_signature))
concreto_specifico = len(sharpener_words_set.intersection(sharpener_signature)) / min(len(sharpener_words_set),
                                                                                    len(sharpener_signature))
# Stampa dei risultati di similarità
print("Astratto - Generico: ", astratto_generico)
print("Astratto - Specifico: ", astratto_specifico)
print("Concreto - Generico: ", concreto_generico)
print("Concreto - Specifico: ", concreto_specifico)

Astratto - Generico:  0.2025916803503011
Astratto - Specifico:  0.12072796934865905
Concreto - Generico:  0.2811384783798581
Concreto - Specifico:  0.37766283524904154

Con utilizzo di gloss e esempi di WordNet:
Astratto - Generico:  0.375
Astratto - Specifico:  0.375
Concreto - Generico:  0.5
Concreto - Specifico:  0.5


#Esercizio 1.2
Stampiamo le 3 parole più usate dalle definizioni di ogni concetto
per provare a dare una spiegazione ai valori di similarità

In [ ]:
# Data la lista delle parole, la ritorna ordinata in base alla frequenza
def get_ordered_list_words(definizioni):
    common_words = []
   
    fdist1 = FreqDist(token.lower() for word in definizioni 
                      for token in word )
    for tupla in fdist1.most_common():
      lista= list (tupla)
      common_words.append(lista[0])
    return common_words

list_concepts=[courage_definitions,paper_definitions,apprehension_definitions,sharpener_definitions]

# Per ogni concetto da trovare
for c in list_concepts :
  # Si prendono tutti i termini a partire dalle definizioni scritte per il concetto, ordinati per frequenza
  concept_words = get_ordered_list_words(c)
  print(concept_words[:3])

['ability', 'fear', 'face']
['material', 'write', 'use']
['feel', 'anxiety', 'state']
['pencil', 'sharpen', 'use']
